In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")


In [18]:
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY



In [19]:
LANGCHAIN_PROJECT

'langgraph-prerequist'

In [20]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

d:\langgraph-end-to-end\langgraphvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

## Simple AI Assistant

In [16]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

As an AI assistant, I can help you with a variety of tasks, such as:

* **Generating creative content:** I can write stories, poems, articles, and even code.
* **Answering your questions:** I have access to a vast store of knowledge and can provide information on a wide range of topics.
* **Summarizing text:** I can condense large amounts of text into shorter, more manageable summaries.
* **Translating languages:** I can translate text between multiple languages.
* **Providing conversation:** I can engage in natural-sounding conversations on a variety of topics.

What can I do for you today?

The future scope of AI is incredibly vast and exciting, with potential to revolutionize nearly every aspect of our lives. Here's a glimpse into some key areas:

**1. Enhanced Productivity and Automation:**

* **Workplace Automation:** AI will automate repetitive tasks, freeing up human workers for more creative and strategic work.
* **Personalized Learning:** AI-powered tutors and educational plat

In [23]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [24]:
store={}

In [26]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [27]:
config = {"configurable": {"session_id": "firstchat"}}

In [28]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [29]:
model_with_memory.invoke(("Hi! I'm kumar"),config=config).content

"Hello Kumar, it's nice to meet you! My name is Gemini, and I'm a multi-modal AI model, also known as a conversational AI or chatbot trained by Google. I'm designed to understand and respond to a wide range of natural language inputs, and to generate human-like text. I'm still under development, but I'm learning new things every day. I'm excited to chat with you and learn more about you. What would you like to talk about?"

In [30]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'You told me your name is Kumar. Is that correct?'

In [31]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm kumar", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Kumar, it's nice to meet you! My name is Gemini, and I'm a multi-modal AI model, also known as a conversational AI or chatbot trained by Google. I'm designed to understand and respond to a wide range of natural language inputs, and to generate human-like text. I'm still under development, but I'm learning new things every day. I'm excited to chat with you and learn more about you. What would you like to talk about?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}

## RAG with LCEL (LangChain Expression Language)

In [33]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [34]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [35]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [36]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [37]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

**Llama 3 is a large language model developed by Google AI.**

**Here are 3 important points about Llama 3:**

1. It is the 8B parameter version of Llama.
2. It was released in April 2024.
3. It is used by both Google AI Platform and Google Cloud's AlloyDB service.


## Let's Start with Tools and Agents

In [40]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [41]:
api_wrapper=WikipediaAPIWrapper()

In [42]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [43]:
tool.name

'wikipedia'

In [44]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [45]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [46]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.



Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment infor

In [47]:
from langchain_community.tools import YouTubeSearchTool

In [48]:
tool2=YouTubeSearchTool()

In [49]:
tool2.name

'youtube_search'

In [50]:
tool2.run("kumar")

"['https://www.youtube.com/watch?v=jX45FO_6cHU&pp=ygUFa3VtYXI%3D', 'https://www.youtube.com/watch?v=T3vH1Avyl7s&pp=ygUFa3VtYXI%3D']"

In [51]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [52]:
tool3=TavilySearchResults()

ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
tool.invoke({"query": "What happened in the latest burning man floods"})

In [62]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [63]:
tool=load_tools(["wikipedia"],llm=llm)

In [64]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\swaro\AppData\Local\Temp\ipykernel_8900\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [65]:
agent.run("What is current GDP of India?")

C:\Users\swaro\AppData\Local\Temp\ipykernel_8900\3730032442.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is current GDP of India?")




> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 141th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. India has about 1,900 public sector companies, with the Ind

'I am sorry, I cannot answer this question as the GDP of India is not mentioned in the context.'